In [1]:
import sympy as sp
from sympy import cos, sin
from sympy import init_printing
from IPython.display import Math, display
init_printing(use_latex=True)

def rotx_sym(th):
    R = sp.Matrix([[1, 0, 0],
                  [0, sp.cos(th), -sp.sin(th)],
                  [0, sp.sin(th), sp.cos(th)]])
    return R
def roty_sym(th):
    R = sp.Matrix([[sp.cos(th), 0, sp.sin(th)],
                   [0, 1, 0],
                   [-sp.sin(th), 0, sp.cos(th)]])
    return R
def rotz_sym(th):
    R = sp.Matrix([[sp.cos(th), -sp.sin(th), 0],
                   [sp.sin(th), sp.cos(th), 0],
                   [0, 0, 1]])
    return R

# making a symbolic SE3 function
def se3_sym(R = sp.eye(3), p = sp.Matrix([[0], [0], [0]])):

    T = sp.eye(4)
    T[0:3, 0:3] = R
    T[0:3, 3] = p
    
    return T

def get_A_sym(dh, jt_type = 'r', q=sp.Symbol('q')):
    if jt_type == 'r':
        A = se3_sym(R = rotz_sym(dh[0]+q)) @\
            se3_sym(p = sp.Matrix([[0], [0], [dh[1]]])) @\
            se3_sym(p = sp.Matrix([[dh[2]], [0], [0]])) @\
            se3_sym(R = rotx_sym(dh[3]))
    elif jt_type == 'p':
        A = se3_sym(R = rotz_sym(dh[0])) @\
            se3_sym(p = sp.Matrix([[0], [0], [dh[1]+q]])) @\
            se3_sym(p = sp.Matrix([[dh[2]], [0], [0]])) @\
            se3_sym(R = rotx_sym(dh[3]))
    else:
        A = None

    return A

In [2]:
# 2 (a) =======================================================================
q1, q2, d1, a2 = sp.symbols('q_1, q_2, d_1, a_2')

dh_param = [[0, d1, 0, sp.pi/2.0],
            [0, 0, a2, 0]]

T_01 = get_A_sym(dh_param[0], 'r', q1)
T_12 = get_A_sym(dh_param[1], 'r', q2)
T_02 = T_01 @ T_12
T = [T_01, T_02]

display(sp.trigsimp(T_02.evalf()))

# Now get the Jacobian
# Axis of rotation and position of joint 1
z0 = sp.Matrix([[0],[0],[1]])
p0 = sp.Matrix([[0],[0],[0]])

# Axis of rotation and position of joint 2
z1 = T_01[:3, 2]
p1 = T_01[:3, 3]
# Position of end effector
p2 = T_02[:3, 3]

J = sp.zeros(6,2)

# Jv and Jw for joint 1
J[:3,0] = z0.cross(p2-p0)
J[3:,0] = z0

# Jv and Jw for joint 2
J[:3,1] = z1.cross(p2-p1)
J[3:,1] = z1


print("FINAL J:")
display(J)

⎡cos(q₁)⋅cos(q₂)  -sin(q₂)⋅cos(q₁)  sin(q₁)   a₂⋅cos(q₁)⋅cos(q₂)⎤
⎢                                                               ⎥
⎢sin(q₁)⋅cos(q₂)  -sin(q₁)⋅sin(q₂)  -cos(q₁)  a₂⋅sin(q₁)⋅cos(q₂)⎥
⎢                                                               ⎥
⎢    sin(q₂)          cos(q₂)          0       a₂⋅sin(q₂) + d₁  ⎥
⎢                                                               ⎥
⎣       0                0             0             1.0        ⎦

FINAL J:


⎡-a₂⋅sin(q₁)⋅cos(q₂)             -a₂⋅sin(q₂)⋅cos(q₁)           ⎤
⎢                                                              ⎥
⎢a₂⋅cos(q₁)⋅cos(q₂)              -a₂⋅sin(q₁)⋅sin(q₂)           ⎥
⎢                                                              ⎥
⎢                           2                     2            ⎥
⎢         0           a₂⋅sin (q₁)⋅cos(q₂) + a₂⋅cos (q₁)⋅cos(q₂)⎥
⎢                                                              ⎥
⎢         0                            sin(q₁)                 ⎥
⎢                                                              ⎥
⎢         0                           -cos(q₁)                 ⎥
⎢                                                              ⎥
⎣         1                               0                    ⎦

In [7]:
# 2 (b) ===========================================================
import sys
sys.path.append("/home/daniel/software/MEEN537/")
import kinematics as kin
from visualization import VizScene
import sympy as sp
import numpy as np
import time
np.set_printoptions(precision=4, suppress=True)

dh = [[0, 0.3, 0, np.pi/2.0],
      [0, 0, 0.3, 0]]

arm = kin.SerialArm(dh)

# defining two different sets of joint angles
q_set1 = [0, 0]
q_set2 = [np.pi/4, np.pi/4]

# calculating two different jacobians for the two different joint configurations. 
J1 = arm.jacob(q_set1)
J2 = arm.jacob(q_set2)

print("from first set of q's, J is:")
sp.pprint(J1)
print("from second set of q's, J is:")
sp.pprint(J2)

from first set of q's, J is:
⎡ 0            0          ⎤
⎢                         ⎥
⎢0.3  1.83697019872103e-17⎥
⎢                         ⎥
⎢ 0           0.3         ⎥
⎢                         ⎥
⎢ 0            0          ⎥
⎢                         ⎥
⎢ 0           -1.0        ⎥
⎢                         ⎥
⎣ 1   6.12323399573677e-17⎦
from second set of q's, J is:
⎡-0.15         -0.15        ⎤
⎢                           ⎥
⎢0.15          -0.15        ⎥
⎢                           ⎥
⎢  0     0.212132034355964  ⎥
⎢                           ⎥
⎢  0     0.707106781186547  ⎥
⎢                           ⎥
⎢  0     -0.707106781186548 ⎥
⎢                           ⎥
⎣  1    6.12323399573677e-17⎦


In [8]:
# 3 =======================================================================
q1, q2, a1, ac = sp.symbols('q_1, q_2, a_1, a_c')

dh_param = [[0, 0, a1, 0],
            [0, 0, ac, 0]]

T_01 = get_A_sym(dh_param[0], 'r', q1)
T_12 = get_A_sym(dh_param[1], 'r', q2)
T_02 = T_01 @ T_12

display(sp.trigsimp(T_02.evalf()))

# Now get the Jacobian
# Axis of rotation and position of joint 1
z0 = sp.Matrix([[0],[0],[1]])
p0 = sp.Matrix([[0],[0],[0]])

# Axis of rotation and position of joint 2
z1 = T_01[:3, 2]
p1 = T_01[:3, 3]
# Position of end effector
p2 = T_02[:3, 3]

J = sp.zeros(6,2)

# Jv and Jw for joint 1
J[:3,0] = z0.cross(p2-p0)
J[3:,0] = z0

# Jv and Jw for joint 2
J[:3,1] = z1.cross(p2-p1)
J[3:,1] = z1


print("FINAL J:")
display(sp.trigsimp(J.evalf()))

⎡cos(q₁ + q₂)  -sin(q₁ + q₂)   0   a₁⋅cos(q₁) + a_c⋅cos(q₁ + q₂)⎤
⎢                                                               ⎥
⎢sin(q₁ + q₂)  cos(q₁ + q₂)    0   a₁⋅sin(q₁) + a_c⋅sin(q₁ + q₂)⎥
⎢                                                               ⎥
⎢     0              0        1.0                0              ⎥
⎢                                                               ⎥
⎣     0              0         0                1.0             ⎦

FINAL J:


⎡-a₁⋅sin(q₁) - a_c⋅sin(q₁ + q₂)  -a_c⋅sin(q₁ + q₂)⎤
⎢                                                 ⎥
⎢a₁⋅cos(q₁) + a_c⋅cos(q₁ + q₂)   a_c⋅cos(q₁ + q₂) ⎥
⎢                                                 ⎥
⎢              0                         0        ⎥
⎢                                                 ⎥
⎢              0                         0        ⎥
⎢                                                 ⎥
⎢              0                         0        ⎥
⎢                                                 ⎥
⎣             1.0                       1.0       ⎦